In [236]:
# 1-1

import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_1_1.csv')
# print(df.info())
# print(df.head())


cond1 = (df['ProductA가격'] != 0)
cond2 = (df['ProductB가격'] != 0)
df_filtered = df[cond1 & cond2]
# print(df_filtered)
df_filtered['가격차이'] = df_filtered['ProductA가격'] - df_filtered['ProductB가격']
answer = round(df_filtered.groupby('도시명')['가격차이'].mean().max(), 1)
print(answer)

# 1-2

import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_1_2.csv')
#print(df.info())
#print(df.head())

df['Height_m^2'] = (df['Height_cm'] / 100)**2
df['BMI'] = round(df['Weight_kg'] / df['Height_m^2'], 1)
df['구분'] = np.where(df['BMI'] >= 25, '비만',
                    np.where(df['BMI'] >= 23, '과체중',
                    np.where(df['BMI'] >= 18.5, '정상',
                    np.where(df['BMI'] < 18.5, '저체중', ''))))

result1 = (df['구분'] == '저체중').sum()
result2 = (df['구분'] == '비만').sum()
print(result1 + result2)

# 1-3

import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_1_3.csv')
#print(df.info())
#print(df.head())

df['순생산량'] = (df['products_made_domestic'] + df['products_made_international']) - (df['products_sold_domestic'] + df['products_sold_international'])
max_production_per_year = df.loc[df.groupby('year')['순생산량'].idxmax()]
result = max_production_per_year['순생산량'].sum()
print(result)

16250.0
74
5788


<ipython-input-236-916114632>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['가격차이'] = df_filtered['ProductA가격'] - df_filtered['ProductB가격']


In [259]:
#2
import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_2_test.csv')

#2-1 (인포)

#print(train.info())
#print(test.info())

#2-2 (데이터 분할)
train_X = train.drop('DBP', axis=1)
train_y = train['DBP']
test_X = test.drop('DBP',axis =1)
test_y = test['DBP']
test_id = test['ID']

#2-3 (독-종 분리)
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size=0.3, random_state=1)

#2-4 (cat, num 분리)
cat_columns = train_X.select_dtypes('object').columns.to_list()
num_columns = train_X.select_dtypes('number').columns.to_list()

#2-5 (전처리)
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
stdscaler = StandardScaler()
onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
si = SimpleImputer(strategy = 'most_frequent')

train_X_std = stdscaler.fit_transform(train_X[num_columns])
valid_X_std = stdscaler.transform(valid_X[num_columns])
test_X_std = stdscaler.transform(test_X[num_columns])

train_X_imputed = si.fit_transform(train_X[cat_columns])
valid_X_imputed = si.transform(valid_X[cat_columns])
test_X_imputed = si.transform(test_X[cat_columns])

train_X_ohe = onehotencoder.fit_transform(train_X_imputed)
valid_X_ohe = onehotencoder.transform(valid_X_imputed)
test_X_ohe = onehotencoder.transform(test_X_imputed)

train_X_preprocessed = np.concatenate([train_X_std, train_X_ohe], axis=1)
valid_X_preprocessed = np.concatenate([valid_X_std, valid_X_ohe], axis=1)
test_X_preprocessed = np.concatenate([test_X_std, test_X_ohe], axis=1)

#2-6 (모델 적합)
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state =1)
rf.fit(train_X_preprocessed, train_y)

#2-7 (검증, 성능확인)
from sklearn.metrics import root_mean_squared_error
pred_val = rf.predict(valid_X_preprocessed)
print(root_mean_squared_error(valid_y, pred_val))

#2-8 (최종예측)
test_pred = rf.predict(test_X_preprocessed)
test_pred = pd.DataFrame(test_pred, columns=['pred'])

#2-9 (생성,저장)
result = pd.concat([test_id, test_pred], axis=1)
result.to_csv('result.csv', index = False)
print(pd.read_csv('result.csv'))

12.396777629867998
       ID   pred
0    2856  73.80
1    3116  80.58
2    1125  76.19
3    3604  78.98
4    1334  76.32
..    ...    ...
124  3841  75.93
125  3003  73.83
126    94  71.55
127  2474  75.51
128  2053  83.61

[129 rows x 2 columns]


In [241]:
#3
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_3_1.csv')
#print(df.info())
#print(df.head())

#3-1-1
work_hours_mean = df['work_hours'].mean()
work_hours_std = df['work_hours'].std()
result1 = round(work_hours_mean, 3)
result2 = round(work_hours_std, 3)

print(result1, result2)

#3-1-2
from scipy.stats import kstest
from scipy.stats import norm

statistic, p_value = kstest(df['work_hours'], 'norm', args = (work_hours_mean, work_hours_std))
print(round(statistic, 3))
print(round(p_value, 3))

#3-1-3
p_value = p_value.round(3)

if p_value < 0.05:
  result = '기각'
else:
  result = '채택'
print(result)

# 3-2-1
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/6_3_2.csv')
# print(df.info())

X = df[['area', 'rooms', 'age']]
y = df['price']
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

coef = model.params[1:]
coef_max = coef.idxmax()
print(coef_max)

# 3-2-2(마지막)

p_values = model.pvalues
print(len(p_values < 0.05)) # 상수항 포함

8.09 1.519
0.064
0.778
채택
rooms
4
